cf. https://www.codewars.com/kata/5b5fe164b88263ad3d00250b (3 kyu) - 2019-12-07

[Alphametics](https://en.wikipedia.org/wiki/Verbal_arithmetic) is a type of cryptarithm (or  cryptarithmetic) in which a set of words is written down in the form of a long addition sum or some other mathematical problem. The objective is to replace the letters of the alphabet with decimal digits to make a valid arithmetic sum.  

For this kata, your objective is to write a function that accepts an alphametic equation in the form of a single-line string and returns a valid arithmetic equation in the form of a single-line string.

**Test Examples**

INPUT: `"SEND + MORE = MONEY"`  
SOLUT: `"9567 + 1085 = 10652"`  
```
  S E N D
+ M O R E
---------
M O N E Y
```

INPUT: `"ELEVEN + NINE + FIVE + FIVE = THIRTY"`  
SOLUT: `"797275 + 5057 + 4027 + 4027 = 810386"`  
```
  E L E V E N 
+     N I N E 
+     F I V E
+     F I V E
-------------
  T H I R T Y
```

Some puzzles may have multiple valid solutions; your function only needs to return one:  
BIG + CAT = LION  
403 + 679 = 1082  
326 + 954 = 1280  
304 + 758 = 1062  
...etc.


**Technical Details**
  
* All alphabetic letters in the input will be uppercase
* Each unique letter may only be assigned to one unique digit (bijective mapping)
* As a corollary to the above, there will be a maximum of 10 unique letters in any given test
* No leading zeroes
* The equations will only deal with addition with multiple summands on the left side   
  (**summands** are LHS, ex: SEND, MORE ...) and one term on the right side
* The number of summands will range between 2 and 7, inclusive
* The length of each summand will range from 2 to 8 characters, inclusive
* All test cases will be valid and will have one or more possible solutions
* *Optimize your code* -- a naive, brute-force algorithm may **time out** before the first test completes

**Resolution** 

```
  S E N D
+ M O R E
---------
M O N E Y
```

* $M is 1$, obviously (it cannot be 0)
* $D + E = Y + 10.c_1$ 
* $c_1 + N + R = E + 10.c_2$
* $c_2 + E + O = N + 10.c_3$
* $c_3 + S + M = O + 10.c_4$
* $c_4 = M  \equiv 1 = M$ 

$\color{blue}{M = 1}$
    
System can be rewritten as follows:
* $(i) \space D + E = Y + 10.c_1$ 
* $(ii) \space c_1 + N + R = E + 10.c_2$
* $(iii) \space c_2 + E + O = N + 10.c_3$
* $(iv) \space c_3 + S + 1 = O + 10 \equiv c_3 + S = O + 9$

From (iv) $\space c_3 + S = O + 9 \Rightarrow (c_3=0 \land S=9, O=0) \lor (c_3=1 \Rightarrow S = O + 8 \Rightarrow S \ge 8, O \lt 2) => O=1 \Rightarrow \varnothing$ ($M=1$ already).  
Hence $\color{blue}{O = 0}$   

From (iii) $r_2 + E + 0 = N + 10.r_3 \Rightarrow r_2 + E = N + 10.r_3$  
$r_2 + E = N + 10.r_3$, if $r_3 = 1 \Rightarrow r_2 + E = N + 10$, 2 cases:
* $r_2 = 0, \space 0 + E = N + 10 \equiv E = N + 10$, this is impossible
* $r_2 = 1, \space 1 + E = N + 10 \equiv E = N + 9 => N=0 \land E=9$, this is impossible, because $O=0$

Hence $r_3 = 0 \land \color{blue}{S=9}$  

From (iii): $r_2 + E = N$, 2 cases:
* $r_2 = 0, \space E = N$, impossible
* $r_2 = 1, \space 1 + E = N$

Hence $r_2 = 1$

From (ii): $r_1 + N + R = E + 10 \equiv r_1 + N + R = N - 1 + 10 \equiv r_1 + R = 9$, given $S = 9$:  
$r_1 = 1 \land \color{blue}{R = 8}$

From (i), $D + E = Y + 10$ and $1 + E = N$:
* $Y \ge 1$
* We know maximum $D + E = 13$ (as 9, 8 are already taken, so D, E would be (6, 7) or (7, 6).
* But $E = 7 \Rightarrow N = 8$, impossible
* And $E = 6 \Rightarrow N = 7 \land D = 7$, impossible

So:
* E is 5, 4, ~3, 2, 1~ and  
* N is 6, 5, ~4, 3, 2~ and  
* D 7, 6 and  
* Y = (7 + 5 - 10 =) 2, ~(6 + 4 -10) 0~  

Hence $\color{blue}{Y = 2}$ (only possibility) $\Rightarrow \color{blue}{E =5} \land \color{blue}{N=6}$
 

Some more examples:  
* `  D A Y S  +     T O O =   S H O R T`
* `    T W O  +   D A Y S =     M O R E`
* `  W A I T  +     A L L =   G I F T S`
* `  N I N E  +   F I V E =   W I V E S`
* `  E V E R  + S I N C E = D A R W I N` 
* `    U S A  +   U S S R =   P E A C E`
* `P O I N T  +   Z E R O = E N E R G Y`
* `      G O  +       T O =       O U T`

* ` E A T  +  E A T  +  E A T = B E E T`

In [1]:
using Test
using BenchmarkTools

In [2]:
const ALL_DIGITS = [d for d in 0:9]


_input = "SEND + MORE = MONEY"
_s, _result = map(strip, split(_input, '='))

2-element Array{SubString{String},1}:
 "SEND + MORE"
 "MONEY"      

In [3]:
_summands = map(strip, split(_s, "+"))

2-element Array{SubString{String},1}:
 "SEND"
 "MORE"

In [7]:
function words_result(str::String)
    s, result = map(strip, split(str, '='))
    summands = map(strip, split(s, "+"))
    return (result=split(result, ""), 
            summands=map(s -> split(s, ""), summands))
end

words_result (generic function with 1 method)

In [8]:
words_result("SEND + MORE = MONEY")

(result = SubString{String}["M", "O", "N", "E", "Y"], summands = Array{SubString{String},1}[["S", "E", "N", "D"], ["M", "O", "R", "E"]])

In [9]:
function wr_tuple_rev(wr_tuple)
    """
    ```jldoctest
    julia> wr_tuple = (result = ["M", "O", "N", "E", "Y"], summands = [["S", "E", "N", "D"], ["M", "O", "R", "E"]])
    julia> wr_tuple_rev(wr_tuple)
    (result = ["Y", "E", "N", "O", "M"], summands = [["D", "N", "E", "S"], ["E", "R", "O", "M"])
    ```
    """
    res = reverse(wr_tuple.result)
    sums = map(reverse, wr_tuple.summands)
    return (result=res, summands=sums)
end

wr_tuple_rev (generic function with 1 method)

In [10]:
# wr_tuple = (result = "MONEY", summands = ["SEND", "MORE"])
wr_tuple = (result = ["M", "O", "N", "E", "Y"], summands = [["S", "E", "N", "D"], ["M", "O", "R", "E"]])

wr_tuple_rev(wr_tuple)

(result = ["Y", "E", "N", "O", "M"], summands = Array{String,1}[["D", "N", "E", "S"], ["E", "R", "O", "M"]])

In [11]:
wr_tuple1 = (result = ["M", "O", "N", "E", "Y"], summands = [["S", "E", "N", "D"], ["M", "O", "R", "E"]])
@test wr_tuple_rev(wr_tuple1) == (result = ["Y", "E", "N", "O", "M"], summands = [["D", "N", "E", "S"], ["E", "R", "O", "M"]])

wr_tuple2 = (result = ["T", "H", "I", "R", "T", "Y"], summands = [["E", "L", "E", "V", "E", "N"], ["N", "I", "N", "E"], 
        ["F", "I", "V", "E"], ["F", "I", "V", "E"]])
@test wr_tuple_rev(wr_tuple2) == (result = ["Y", "T", "R", "I", "H", "T"], summands = 
    [["N", "E", "V", "E", "L", "E"], ["E", "N", "I", "N"], ["E", "V", "I", "F"], ["E", "V", "I", "F"]])

Test Passed

In [12]:
function all_letters(str::String)
    nstr = replace(str, ['+', ' ', '='] => "")
    letters = split(nstr, "")
    sort!(letters)   # in-place
    unique!(letters) # in-place 
    return letters
end

all_letters (generic function with 1 method)

In [13]:
all_letters("SEND + MORE = MONEY")

8-element Array{SubString{String},1}:
 "D"
 "E"
 "M"
 "N"
 "O"
 "R"
 "S"
 "Y"

In [14]:
@test all_letters("SEND + MORE = MONEY") == ["D", "E", "M", "N", "O", "R", "S", "Y"]
@test all_letters("ELEVEN + NINE + FIVE + FIVE = THIRTY") == ["E", "F", "H", "I", "L", "N", "R", "T", "V", "Y"]

Test Passed

In [15]:
mutable struct AExpr
    lhs::String
    rhs::String
end

In [16]:
Base.show(io::IO, aexpr::AExpr) = print(io, "lhs: $(aexpr.lhs) = rhs: $(aexpr.rhs)")

In [17]:
function gen_constraints(wr_tuple_rev)::Tuple{Array{AExpr, 1}, Array{String, 1}}
    len_res = length(wr_tuple_rev.result)
    constraints = []
    ix, aexpr = 1, AExpr("", "") 
    symbols = [] # Array{String, 1}[]
    
    while true  
        # lhs
        for ary in wr_tuple_rev.summands
            if ix <= length(ary)
                ary[ix] in symbols || push!(symbols, ary[ix])
                
                if isempty(aexpr.lhs)
                    aexpr.lhs = "$(ary[ix])"
                else
                    aexpr.lhs = "$(aexpr.lhs) + $(ary[ix])"
                end
            end
        end
        
        # rhs
        wr_tuple_rev.result[ix] in symbols || push!(symbols, wr_tuple_rev.result[ix])
        aexpr.rhs =
            if ix == length(wr_tuple_rev.result)
                "$(wr_tuple_rev.result[ix])"
            else
                # carry
                "$(wr_tuple_rev.result[ix]) + 10 * c_$(ix)"
            end
        push!(constraints, aexpr)
        
        # next 
        ix += 1
        ix > len_res && break
        
        # next carry
        aexpr = AExpr("c_$(ix - 1)", "")
        push!(symbols, "c_$(ix - 1)")
    end
    
    return (constraints, symbols)
end

gen_constraints (generic function with 1 method)

In [18]:
wr_tuple_rev1 = wr_tuple_rev(wr_tuple1)

(result = ["Y", "E", "N", "O", "M"], summands = Array{String,1}[["D", "N", "E", "S"], ["E", "R", "O", "M"]])

In [19]:
constraint_ary, ary_symbols = gen_constraints(wr_tuple_rev1)

(AExpr[lhs: D + E = rhs: Y + 10 * c_1, lhs: c_1 + N + R = rhs: E + 10 * c_2, lhs: c_2 + E + O = rhs: N + 10 * c_3, lhs: c_3 + S + M = rhs: O + 10 * c_4, lhs: c_4 = rhs: M], ["D", "E", "Y", "c_1", "N", "R", "c_2", "O", "c_3", "S", "M", "c_4"])

Remarks:  
* $\forall i, c_i ∈ [0, 1]$  
* $S \neq 0 \land M \neq 0$

In [20]:
ary_symbols

12-element Array{String,1}:
 "D"  
 "E"  
 "Y"  
 "c_1"
 "N"  
 "R"  
 "c_2"
 "O"  
 "c_3"
 "S"  
 "M"  
 "c_4"

In [21]:
const ETYPES = (:carry, :var)  # Symbol Expression type either :carry or :var
typeof(ETYPES)

Tuple{Symbol,Symbol}

In [22]:
mutable struct Elem
    symbol::String
    value::Union{Nothing, Int}
    pvalues::Set{Int}
    etype::Symbol
end

In [23]:
hsymbols = Dict{String, Elem}()

for k in ary_symbols
    cond = match(r"\Ac_\d+", k) != nothing
    hsymbols[k] = Elem(
        k, 
        nothing, 
        cond ? Set([0, 1]) : Set(collect(0:9)),
        cond ? ETYPES[1] : ETYPES[2] # ETYPES][1] == :carry, ETYPES][2] == :var
    )
end

hsymbols

Dict{String,Elem} with 12 entries:
  "c_4" => Elem("c_4", nothing, Set([0, 1]), :carry)
  "O"   => Elem("O", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "M"   => Elem("M", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "N"   => Elem("N", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "c_1" => Elem("c_1", nothing, Set([0, 1]), :carry)
  "c_2" => Elem("c_2", nothing, Set([0, 1]), :carry)
  "D"   => Elem("D", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "E"   => Elem("E", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "Y"   => Elem("Y", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "S"   => Elem("S", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "R"   => Elem("R", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "c_3" => Elem("c_3", nothing, Set([0, 1]), :carry)

In [24]:
symbols = Dict(k => nothing for k in keys(hsymbols)) # short version

Dict{String,Nothing} with 12 entries:
  "c_4" => nothing
  "O"   => nothing
  "M"   => nothing
  "N"   => nothing
  "c_1" => nothing
  "c_2" => nothing
  "D"   => nothing
  "E"   => nothing
  "Y"   => nothing
  "S"   => nothing
  "R"   => nothing
  "c_3" => nothing

In [25]:
function gen_constraints_as_expr(ary::Array{AExpr, 1})::Array{NamedTuple{(:lhs, :rhs),T} where T<:Tuple,1}
    return map(aexpr -> (lhs=Meta.parse(aexpr.lhs), rhs=Meta.parse(aexpr.rhs)), ary)
end

gen_constraints_as_expr (generic function with 1 method)

In [28]:
const AEXPR_LST = gen_constraints_as_expr(constraint_ary)

5-element Array{NamedTuple{(:lhs, :rhs),T} where T<:Tuple,1}:
 (lhs = :(D + E), rhs = :(Y + 10c_1))      
 (lhs = :(c_1 + N + R), rhs = :(E + 10c_2))
 (lhs = :(c_2 + E + O), rhs = :(N + 10c_3))
 (lhs = :(c_3 + S + M), rhs = :(O + 10c_4))
 (lhs = :c_4, rhs = :M)                    

### Evaluation

In [29]:
function eval_aexpr(aexpr_lst::Array{NamedTuple{(:lhs, :rhs),T} where T<:Tuple,1}, d_env)
    # set the env
    [@eval $(Symbol("$(k)")) = ($v) for (k, v) in d_env]
    
    for aexpr in aexpr_lst
        try
            lhs = eval(aexpr.lhs)
            rhs = eval(aexpr.rhs)
        
            if (lhs != rhs) 
                println("eval: $(aexpr.lhs)[==$(lhs)] != $(aexpr.rhs)[==$(rhs)] - false / stop")
                return false
            else
                println("eval: $(aexpr.lhs)[==$(lhs)] == $(aexpr.rhs)[==$(rhs)] - true / continue")
            end
            
        catch ex
            println("[!] lhs: $(aexpr.lhs) / rhs: $(aexpr.rhs) - not eval-uable")
            continue
        end
    end
    
    return true
end

eval_aexpr (generic function with 1 method)

In [30]:
eval_aexpr(AEXPR_LST, 
    merge(hsymbols, Dict("D" => 9, "E" => 2, "Y" => 3, "c_1" => 0)))

eval: D + E[==11] != Y + 10c_1[==3] - false / stop


false

In [31]:
eval_aexpr(AEXPR_LST, 
    merge(hsymbols, Dict("D" => 9, "E" => 2, "Y" => 1, "c_1" => 1)))

eval: D + E[==11] == Y + 10c_1[==11] - true / continue
[!] lhs: c_1 + N + R / rhs: E + 10c_2 - not eval-uable
[!] lhs: c_2 + E + O / rhs: N + 10c_3 - not eval-uable
[!] lhs: c_3 + S + M / rhs: O + 10c_4 - not eval-uable
[!] lhs: c_4 / rhs: M - not eval-uable


true

In [32]:
eval_aexpr(AEXPR_LST, 
    merge(hsymbols, Dict("c_4" => 1, "M" => 1)))

[!] lhs: D + E / rhs: Y + 10c_1 - not eval-uable
[!] lhs: c_1 + N + R / rhs: E + 10c_2 - not eval-uable
[!] lhs: c_2 + E + O / rhs: N + 10c_3 - not eval-uable
[!] lhs: c_3 + S + M / rhs: O + 10c_4 - not eval-uable
eval: c_4[==1] == M[==1] - true / continue


true

In [33]:
eval_aexpr(AEXPR_LST, 
    merge(hsymbols, Dict("c_4" => 1, "M" => 2, "D" => 3, "E" => 1, "Y" => 5, "c_1" => 0)))

eval: D + E[==4] != Y + 10c_1[==5] - false / stop


false

In [34]:
eval_aexpr(AEXPR_LST, 
    merge(shymbols, Dict("c_4" => 1, "M" => 2, "D" => 4, "E" => 1, "Y" => 5, "c_1" => 0)))

eval: D + E[==5] == Y + 10c_1[==5] - true / continue
[!] lhs: c_1 + N + R / rhs: E + 10c_2 - not eval-uable
[!] lhs: c_2 + E + O / rhs: N + 10c_3 - not eval-uable
[!] lhs: c_3 + S + M / rhs: O + 10c_4 - not eval-uable
eval: c_4[==1] != M[==2] - false / stop


false

In [35]:
eval_aexpr(AEXPR_LST, 
    merge(hsymbols, Dict("c_4" => 1, "M" => 1, "D" => 5, "E" => 2, "Y" => 7, "c_1" => 0)))

eval: D + E[==7] == Y + 10c_1[==7] - true / continue
[!] lhs: c_1 + N + R / rhs: E + 10c_2 - not eval-uable
[!] lhs: c_2 + E + O / rhs: N + 10c_3 - not eval-uable
[!] lhs: c_3 + S + M / rhs: O + 10c_4 - not eval-uable
eval: c_4[==1] == M[==1] - true / continue


true

### System

Possible Strategy:
* Start with equation with less constraints (in term of length and number of constraints
* LHS (1 symb) = RHS (1 symb) and LHS $\neq$ RHS $\Rightarrow$ solution = LHS $\cap$ RHS 

Constraints:
* Leftmost symbols cannot be 0
* All Symbols have different values

In [36]:
AEXPR_LST

5-element Array{NamedTuple{(:lhs, :rhs),T} where T<:Tuple,1}:
 (lhs = :(D + E), rhs = :(Y + 10c_1))      
 (lhs = :(c_1 + N + R), rhs = :(E + 10c_2))
 (lhs = :(c_2 + E + O), rhs = :(N + 10c_3))
 (lhs = :(c_3 + S + M), rhs = :(O + 10c_4))
 (lhs = :c_4, rhs = :M)                    

In [43]:
t = AEXPR_LST[1]
typeof(t.lhs)   # Expr
t.lhs.head      # :call
t.lhs.args      # 3-element Array{Any,1}:
# :+
# :D
# :E
t.lhs.args[2:end] # 2-element Array{Any,1}:
# :D
# :E
t.rhs.args[2:end] # 2-element Array{Any,1}:
# :Y
# :(10c_1)

2-element Array{Any,1}:
 :Y      
 :(10c_1)

In [44]:
cat(t.lhs.args[2:end], t.rhs.args[2:end]; dims=1)

4-element Array{Any,1}:
 :D      
 :E      
 :Y      
 :(10c_1)

In [45]:
vcat(t.lhs.args[2:end], t.rhs.args[2:end])

4-element Array{Any,1}:
 :D      
 :E      
 :Y      
 :(10c_1)

In [46]:
t_last = aexpr_lst[end]
typeof(t_last.lhs) # Symbol
t_last.lhs         # :c_4

:c_4

In [47]:
function find_eqn_with_less_contraints(aexpr_lst)
    """
    Find equation in given aexpr_lst, which has the less element(s)
    returns first equation if all equations have same number of elements (or members) 
    """
    ary, nmin = fill(0, 3), 10_000_000
    
    for (ix, t) in enumerate(aexpr_lst)
        _ary = fill(ix, 3)
        for (jx, ths) in enumerate(t)
            hs = 
                if typeof(ths) == Expr
                    length(ths.args[2:end])
                elseif typeof(ths) == Symbol
                    1
                end
            _ary[jx + 1] = hs
        end    
        n = _ary[2] + _ary[end]
        if n < nmin
            nmin = n
            for i in 1:3
                ary[i] = _ary[i]
            end
        end
    end
    return ary
end

find_eqn_with_less_contraints (generic function with 1 method)

In [49]:
ary = find_eqn_with_less_contraints(AEXPR_LST)  # [ix, len(lhs), len(rhs)], here [5, 1, 1]

3-element Array{Int64,1}:
 5
 1
 1

In [50]:
# wr_tuple = (result = ["M", "O", "N", "E", "Y"], summands = [["S", "E", "N", "D"], ["M", "O", "R", "E"]])
function non_leading_0!(hsymbols, wr_tuple)
    sym = wr_tuple.result[1]
    hsymbols[sym].pvalues = Set(collect(1:9))
    
    for s in wr_tuple.summands
        hsymbols[s[1]].pvalues = Set(collect(1:9))
    end
    
    hsymbols
end

non_leading_0! (generic function with 1 method)

In [51]:
non_leading_0!(hsymbols, wr_tuple) # side -effect hsymbols

Dict{String,Elem} with 12 entries:
  "c_4" => Elem("c_4", nothing, Set([0, 1]), :carry)
  "O"   => Elem("O", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "M"   => Elem("M", nothing, Set([7, 4, 9, 2, 3, 5, 8, 6, 1]), :var)
  "N"   => Elem("N", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "c_1" => Elem("c_1", nothing, Set([0, 1]), :carry)
  "c_2" => Elem("c_2", nothing, Set([0, 1]), :carry)
  "D"   => Elem("D", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "E"   => Elem("E", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "Y"   => Elem("Y", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "S"   => Elem("S", nothing, Set([7, 4, 9, 2, 3, 5, 8, 6, 1]), :var)
  "R"   => Elem("R", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6, 1]), :var)
  "c_3" => Elem("c_3", nothing, Set([0, 1]), :carry)

In [54]:
# (AExpr[lhs: D + E = rhs: Y + 10 * c_1, 
#        lhs: c_1 + N + R = rhs: E + 10 * c_2, 
#        lhs: c_2 + E + O = rhs: N + 10 * c_3, 
#        lhs: c_3 + S + M = rhs: O + 10 * c_4, 
#        lhs: c_4 = rhs: M], 
#
#  ["D", "E", "Y", "c_1", "N", "R", "c_2", "O", "c_3", "S", "M", "c_4"])

function lhs_rhs_singleton!(hsymbols, aexpr_lst)
    ix, n_l, n_r = find_eqn_with_less_contraints(aexpr_lst)
    
    if n_l == n_r && n_r == 1 # we have 1 equation with singleton element on both side, simplify! 
        lhs_sym = aexpr_lst[ix].lhs
        rhs_sym = aexpr_lst[ix].rhs
        set = intersect(hsymbols[String(lhs_sym)].pvalues, hsymbols[String(rhs_sym)].pvalues)    
        hsymbols[String(lhs_sym)].pvalues = set
        hsymbols[String(rhs_sym)].pvalues = set
        
        if length(set) == 1
            hsymbols[String(lhs_sym)].value = first(set)
            hsymbols[String(rhs_sym)].value = first(set)            
            # this value needs to be remove from all pvalues of all remaining variables
            
            for sym in keys(hsymbols)
                hsymbols[sym].etype == :carry && continue # ignore carries
                hsymbols[sym].pvalues = setdiff(hsymbols[sym].pvalues, set)
            end
        end
    end
end

lhs_rhs_singleton! (generic function with 1 method)

In [55]:
lhs_rhs_singleton!(hsymbols, AEXPR_LST)
hsymbols

Dict{String,Elem} with 12 entries:
  "c_4" => Elem("c_4", 1, Set([1]), :carry)
  "O"   => Elem("O", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6]), :var)
  "M"   => Elem("M", 1, Set(Int64[]), :var)
  "N"   => Elem("N", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6]), :var)
  "c_1" => Elem("c_1", nothing, Set([0, 1]), :carry)
  "c_2" => Elem("c_2", nothing, Set([0, 1]), :carry)
  "D"   => Elem("D", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6]), :var)
  "E"   => Elem("E", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6]), :var)
  "Y"   => Elem("Y", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6]), :var)
  "S"   => Elem("S", nothing, Set([7, 4, 9, 2, 3, 5, 8, 6]), :var)
  "R"   => Elem("R", nothing, Set([0, 4, 7, 9, 2, 3, 5, 8, 6]), :var)
  "c_3" => Elem("c_3", nothing, Set([0, 1]), :carry)

#### Evaluation given context

In [61]:
eval_aexpr(
    AEXPR_LST,
    Dict(k => hsymbols[k].value for k in keys(hsymbols))
)
# merge(symbols, Dict("D" => 9, "E" => 2, "Y" => 3, "c_1" => 0)))

[!] lhs: D + E / rhs: Y + 10c_1 - not eval-uable
[!] lhs: c_1 + N + R / rhs: E + 10c_2 - not eval-uable
[!] lhs: c_2 + E + O / rhs: N + 10c_3 - not eval-uable
[!] lhs: c_3 + S + M / rhs: O + 10c_4 - not eval-uable
eval: c_4[==1] == M[==1] - true / continue


true